## Importação de bibliotecas e pastas

In [1]:
import pandas as pd
import os
import pathlib
import sqlite3
import datetime as dt

In [2]:
from google.colab import drive
drive.mount('/ShardDrives/')

Mounted at /ShardDrives/


In [3]:
pd.set_option('float_format', '{:,.2f}'.format)

## Renomeando arquivos 
**Desnecessário para estes arquivos** 

Executado e nomes alterados

In [ ]:
"""folder = "/ShardDrives/MyDrive/rest_data_extraction /volume_vendas_produto/"
for file in os.listdir(folder):
    old_name = folder + file
    data = pd.read_excel(old_name)
    if file[:7] != 'vendas_':
      report_date = data['Unnamed: 1'][1][-10:]
      new_name = f"{folder}vendas_{report_date}.xlsx"
      os.rename(old_name, new_name)"""

'folder = "/ShardDrives/MyDrive/rest_data_extraction /volume_vendas_produto/"\nfor file in os.listdir(folder):\n    old_name = folder + file\n    data = pd.read_excel(old_name)\n    if file[:7] != \'vendas_\':\n      report_date = data[\'Unnamed: 1\'][1][-10:]\n      new_name = f"{folder}vendas_{report_date}.xlsx"\n      os.rename(old_name, new_name)'

## Data opening

In [4]:
file = "/ShardDrives/MyDrive/rest_data_extraction /base_de_dados/volume_vendas_2018_2022.xlsx"
data = pd.read_excel(file)

## Visão geral do dataframe

In [5]:
data.head(10)

,Evolução de Vendas por Produto,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Séries: Todas,Data de Início: 01-01-2018,Data de Fim: 25-08-2022,NaN,NaN,NaN,NaN
5,Familia: Todas,Sub-Familia: Todas,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Data,Código,Produto,Familia / Sub-Familia,Quantidade,Valor Total S/IVA,Valor Total
8,2018-01-03 00:00:00,100002,Avenida,Bebidas / Cerveja,4,13.90,17.10
9,2018-01-03 00:00:00,100003,Batata Doce,Comida / Evento,9,41.42,46.80


## Limpeza inicial

In [6]:
# Limpar linhas vazias e reiniciar o index

df = data.rename(columns=data.iloc[7]).iloc[8:-1].reset_index(drop=True)
df

,Data,Código,Produto,Familia / Sub-Familia,Quantidade,Valor Total S/IVA,Valor Total
0,2018-01-03 00:00:00,100002,Avenida,Bebidas / Cerveja,4,13.90,17.10
1,2018-01-03 00:00:00,100003,Batata Doce,Comida / Evento,9,41.42,46.80
2,2018-01-03 00:00:00,100005,Berbigão,Comida / Pratos,9,42.83,48.40
3,2018-01-03 00:00:00,100012,Cp Dashi,Bebidas / Soft Drinks,1,3.10,3.50
4,2018-01-03 00:00:00,100022,Cp Phaunus Pet Nat,Bebidas / Espumantes,2,13.01,16
...,...,...,...,...,...,...,...
68219,2022-08-24 00:00:00,101431,Alheta Pescada,Comida / Especiais,4,60.18,68
68220,2022-08-24 00:00:00,101432,Leite Frito,Comida / Sobremesas,6,47.79,54
68221,2022-08-24 00:00:00,101450,Extra Tosta,Comida / Snacks,1,3.54,4
68222,2022-08-24 00:00:00,101454,Cereja,Comida / Sobremesas,1,6.64,7.50


In [7]:
new_column = df['Familia / Sub-Familia'].str.split('/', n=1, expand=True)
df['familia'] = new_column[0]
df['sub_familia'] = new_column[1]
df = df[['Data', 'Código', 'Produto', 'familia', 'sub_familia', 'Quantidade', 'Valor Total S/IVA', 'Valor Total']]
df.columns = ['data', 'codigo', 'produto', 'familia', 'sub_familia', 'quantidade', 'total_s_iva', 'total']

### Tratamento de colunas


In [8]:
# Definindo o tipo adequado para as colunas

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68224 entries, 0 to 68223
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         68224 non-null  object
 1   codigo       68224 non-null  object
 2   produto      68224 non-null  object
 3   familia      68224 non-null  object
 4   sub_familia  68224 non-null  object
 5   quantidade   68224 non-null  object
 6   total_s_iva  68224 non-null  object
 7   total        68224 non-null  object
dtypes: object(8)
memory usage: 4.2+ MB


In [9]:
df.data = pd.to_datetime(df.data)
df['codigo'] = pd.to_numeric(df['codigo'])
df['quantidade'] = pd.to_numeric(df['quantidade'])
df['total_s_iva'] = pd.to_numeric(df['total_s_iva'])
df['total'] = pd.to_numeric(df['total'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68224 entries, 0 to 68223
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   data         68224 non-null  datetime64[ns]
 1   codigo       68224 non-null  int64         
 2   produto      68224 non-null  object        
 3   familia      68224 non-null  object        
 4   sub_familia  68224 non-null  object        
 5   quantidade   68224 non-null  float64       
 6   total_s_iva  68224 non-null  float64       
 7   total        68224 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(3)
memory usage: 4.2+ MB


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [10]:
df.columns

Index(['data', 'codigo', 'produto', 'familia', 'sub_familia', 'quantidade',
       'total_s_iva', 'total'],
      dtype='object')

## Criação do Banco de Dados
**Etapa desnecessária para este caso de já termos o dataset completo e não ser necessário atualizar com novos dados**


In [11]:
#bd = sqlite3.connect("vendas_restaurantes.db")

In [12]:
#cursor = bd.cursor()

### A célula abaixo só é necessário executar na primeira vez, para a criação da tabela no banco de dados

In [13]:
#cursor.execute("CREATE TABLE 'Vendas Restauração 2018_2022' (data INTEGER, codigo INT, produto TEXT, familia TEXT, sub_familia TEXT, quantidade INT, total_s_iva REAL, total REAL)")

In [14]:
#df.head(2)

In [15]:
#for index, row in df.iterrows():
#    data = row[0]
#    codigo = row[1]
#    produto = row[2]
#    familia = row[3]
#    sub_familia = row[4]
#    quantidade = row[5]
#    total_s_iva = row[6]
#    total = row[7]
#    cursor.execute('''INSERT INTO 'Vendas Restauração 2018_2022' \
#                    (data, codigo, produto, familia, sub_familia, quantidade, total_s_iva, total) VALUES (?,?,?,?,?,?,?,?)''', \
#                    (str(data), codigo, produto, familia, sub_familia, quantidade, total_s_iva, total))
#    
#    bd.commit()

##**Primeiras explorações**

In [16]:
# Faturamento total do período

faturamento_total = df.total.sum()
f'A empresa faturou {faturamento_total:,.2f} euros durante o período'

'A empresa faturou 4,603,194.17 euros durante o período'

In [17]:
# Faturamento por família

faturamento_familia = df[['familia', 'total']].groupby(by='familia').sum()
faturamento_familia.sort_values(by='total', ascending=False)

,total
familia,
Comida,"3,058,871.40"
Bebidas,"1,464,298.98"
Outros,"79,387.59"
Talho,370.00
Sem Familia Definida,226.00
Peixaria,20.70
Mercearia,19.50


###Rentabilidade diária

In [18]:
df[df['data'] == '2018-01-03'].count()

data           44
codigo         44
produto        44
familia        44
sub_familia    44
quantidade     44
total_s_iva    44
total          44
dtype: int64

In [19]:
dt.date.isoweekday(df.data[50])

4

In [20]:
# Criando uma lista com todos os dias de venda e atribuindo a uma nova coluna

dia_semana = []
for dia in df.data:
  dia_semana.append(dt.date.isoweekday(dia))

df['dia_semana'] = dia_semana

In [21]:
rentabilidade_diaria = df[['familia', 'total', 'dia_semana']].groupby(by='dia_semana').sum()

In [22]:
# Qual a rentabilidade diária para o período?

rentabilidade_diaria.sort_values(by='total', ascending=False)

,total
dia_semana,
6,"1,089,471.37"
5,"1,082,017.64"
4,"888,494.16"
3,"744,466.71"
7,"719,266.45"
2,"47,654.04"
1,"31,823.80"


In [23]:
# Qual dia da semana possui mais entradas?

df.dia_semana.value_counts()

5    14997
6    14638
4    13709
3    12233
7    11525
2      696
1      426
Name: dia_semana, dtype: int64

##**Regressão Linear - Faturamento futuro**##

In [25]:
# Bibliotecas para construção do modelo

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [26]:
object_df = df.select_dtypes(include=['object']).copy()